In [2]:
from visio_gptq.model import GPTQModel
model = GPTQModel(model_name="TheBloke/wizardLM-7B-GPTQ", device="cuda", wbits=4, groupsize=128)

Loading model ...


/home/marcojoao/mambaforge/envs/vicuna/lib/python3.10/site-packages/safetensors/torch.py:99: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:
/home/marcojoao/mambaforge/envs/vicuna/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/marcojoao/mambaforge/envs/vicuna/lib/python3.10/site-packages/torch/storage.py:899: UserWarning: TypedStorage is deprecated. It will

Done.


In [3]:
from langchain.llms.base import LLM
from typing import Optional, List

class GPTQModelLLM(LLM):
    model: GPTQModel
    params: dict = {}

    @property
    def _llm_type(self) -> str:
        return "gptq_model_llm"

    def callbackFunc(self,txt):
        print(txt, end=" ", flush=True)

    def _call(self, prompt:str, stop: Optional[List[str]] = None) -> str:
        current_stop = self.params.get("stop", [])
        if stop is not None:
            current_stop.extend(stop)
            self.params["stop"] = current_stop

        verbose = self.params.get("verbose", self.verbose)
        output = ""
        generator = model.generate_stream(prompt=prompt, **self.params)
        for result in generator:
            output += " " + result
            if verbose:
                print(result, end=" ", flush=True)
        return output

In [4]:
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate


llm = GPTQModelLLM(model=model, params={"verbose":True, "temperature": 0.7})
DEFAULT_PROMPT = PromptTemplate(
    input_variables=["history","input"],
    template="The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know."
            "Current conversation:"
            "###Human: Give three tips for staying healthy."
            "###AI: Sure, here are three tips for staying healthy:"
            "1. Exercise regularly: Regular physical activity can help improve your overall health and wellbeing. It can also help reduce your risk of chronic conditions such as obesity, diabetes, heart disease, and certain cancers. Aim for at least 150 minutes of moderate-intensity aerobic exercise or 75 minutes of vigorous-intensity aerobic exercise per week, along with muscle-strengthening activities at least two days per week."
            "2. Eat a balanced diet: Eating a balanced diet that is rich in fruits, vegetables, whole grains, lean proteins, and healthy fats can help support your overall health. Try to limit your intake of processed and high-sugar foods, and aim to drink plenty of water throughout the day."
            "3. Get enough sleep: Getting enough quality sleep is essential for your physical and mental health. Adults should aim for seven to nine hours of sleep per night. Establish a regular sleep schedule and try to create a relaxing bedtime routine to help improve the quality of your sleep."
            "{history}"
            "###Human: {input}"
            "###AI:")
conv = ConversationChain(llm=llm, prompt=DEFAULT_PROMPT, verbose=False)

In [5]:
result = conv.predict(input="In Microsoft Excel, how do I replace part of a text from various cells without changing the format of the entire cell?")

To replace part of a text from various cells in Microsoft Excel without changing the format of the entire cell, you can use the "Find and Replace" feature. First, select the cell or range of cells where you want to replace the text. Then, click on the "Find and Replace" button in the "Home" tab of the Excel ribbon. In the "Find What" field, enter the text you want to replace. In the "Replace With" field, enter the new text you want to replace the old text with. Make sure the "Search Format" option is set to "Regular Expressions" and the "Find All" option is checked. Then, click "Replace All" to replace the text in all selected cells. 

In [6]:
from langchain.document_loaders import OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

In [7]:
data = OnlinePDFLoader("https://arxiv.org/pdf/2304.12244v1.pdf").load()
texts = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0).split_documents(data)
docsearch = Chroma.from_documents(texts, embeddings)

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.
Using embedded DuckDB without persistence: data will be transient


In [8]:
query = "write a summary about WizardLM in 100 words"
docs = docsearch.similarity_search(query, include_metadata=True, top_k=5)
chain = load_qa_chain(llm=llm, chain_type="stuff")
result = chain.run(input_documents=docs, question=query, verbose=False)

WizardLM is an AI assistant that empowers large language models to follow complex instructions. It uses Evol-Instruct, a dataset construction method, to improve the effectiveness of language models in various domains. WizardLM has achieved significant results in the experiment, outperforming Alpaca and Vicuna-7b on high-difﬁculty skills. It can perform well on various skills such as writing, math, biology, chemistry, and physics. Overall, WizardLM is an effective tool for solving complex problems with the help of language models. 